# code use to batch format files for spacetx format

In [60]:
import os
from typing import Mapping, Tuple, Union
import numpy as np
from starfish.types import Axes, Coordinates, Features, Number
from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher
from slicedimage import ImageFormat
from skimage.io import imread
from starfish.experiment.builder import write_experiment_json

## define functions

In [61]:
# hakuna matata
if not os.path.isdir(output_dir):
    try:
        os.mkdir(output_dir)
    except:
        os.makedirs(output_dir)


def add_codebook(experiment_json_doc):
    experiment_json_doc['codebook'] = "codebook.json"
    return experiment_json_doc


def make_codebook_json(codebook_csv):
    """ convert color code matrix in csv to json format"""
    codebook_array = []
    with open(codebook_csv, "r") as f:
        for line in f:
            line = line.rstrip('\n').split(',')
            codewords = []
            for r, colorcode in enumerate(line[1:]):
                codewords.append({Axes.ROUND.value: r,
                                  Axes.CH.value: int(colorcode)-1,
                                  Features.CODE_VALUE:1})
            codebook_array.append({Features.CODEWORD:codewords, Features.TARGET: line[0]})
    codebook = Codebook.from_code_array(codebook_array)
    codebook_json_filename = "codebook.json"
    codebook.to_json(os.path.join(output_dir, codebook_json_filename))

def get_tilepos(tilepos_xy_csv):
    tilexy = np.ndarray((0,2))
    with open(tilepos_xy_csv, 'r') as f:
        for line in f:
            line = line.rstrip('\n').split(',')
            tilexy = np.vstack([tilexy, [np.double(line[0]), np.double(line[1])]])
    return tilexy

# TODO: skip reading and writing of images
class ISSTile2D(FetchedTile):
    def __init__(self, file_path, fov):
        self.file_path = file_path
        self.fov = fov

    @property
    def shape(self) -> Tuple[int, ...]:
        return SHAPE

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]]:
        return {
            Coordinates.X: (tilexy[self.fov, 0]*pixelscale, (tilexy[self.fov, 0] + tilesz)*pixelscale),
            Coordinates.Y: (tilexy[self.fov, 1]*pixelscale, (tilexy[self.fov, 1] + tilesz)*pixelscale),
            Coordinates.Z: (0.0, 0.0),
        }

    def tile_data(self) -> np.ndarray:
        return imread(self.file_path)


class ISS2DPrimaryTileFetcher(TileFetcher):
    def __init__(self, path):
        self.path = path

    def get_tile(self, fov: int, r: int, ch: int, z: int) -> FetchedTile:
        return ISSTile2D(os.path.join(self.path, "{}-{}/tile{}.tif".format(FILENAME[r], CHORDER[ch]+1, fov+1)), fov)


class ISS2DAuxTileFetcher(TileFetcher):
    def __init__(self, path, filename_prefix):
        self.path = path
        self.prefix = filename_prefix

    def get_tile(self, fov: int, r: int, ch: int, z: int) -> FetchedTile:
        return ISSTile2D(os.path.join(self.path, self.prefix + "{}.tif".format(fov+1)), fov)




In [72]:
path = 'I:/'
folders_to_remove = ['misc','Seagate','System Volume Information', '$RECYCLE.BIN', '02A','18D','02B','02C','02D','02E','06A','06B','06C','06D','06E','10A','10B','10C','10D','10E','10F']
output = [dI for dI in os.listdir(path) if os.path.isdir(os.path.join(path,dI))]
output_cleaned = [x for x in output if x not in folders_to_remove]
output_cleaned

['14A', '14B', '14C', '14D', '18A', '18B', '18C']

## create folders to output the spacetx formatted data

In [70]:
for i, slide_tag in enumerate(output_cleaned): 
    path = 'H:/'+slide_tag
    if not os.path.exists(path):
        os.mkdir(path)
        
    path1 = 'H:/'+slide_tag+'/SpaceTX_format'
    if not os.path.exists(path1):
        os.mkdir(path1)
    

## rename folders to fit with the format

In [73]:
for i, slide_tag in enumerate(output_cleaned): 
    print(slide_tag)

    # imaging
    #os.rename(src, dst)
    import os
    basedir = 'I:/'+slide_tag+'/preprocessing/ReslicedTiles/'
    for fn in os.listdir(basedir):
        os.rename(os.path.join(basedir,fn), os.path.join(basedir,fn.replace('_ORG','')))

    for fn in os.listdir(basedir):
        os.rename(os.path.join(basedir,fn), os.path.join(basedir,fn.replace('_c','_c-')))

14A
14B
14C
14D
18A
18B
18C


In [76]:
path = 'I:/'
folders_to_remove = ['misc','Seagate','System Volume Information', '$RECYCLE.BIN', '02A','18D', '02B','02C','02D','02E','06A','06B','06C','06D','06E']
output = [dI for dI in os.listdir(path) if os.path.isdir(os.path.join(path,dI))]
output_cleaned = [x for x in output if x not in folders_to_remove]
output_cleaned

['10A',
 '10B',
 '10C',
 '10D',
 '10E',
 '14A',
 '14B',
 '14C',
 '14D',
 '18A',
 '18B',
 '18C']

## main loop

In [77]:
for i, slide_tag in enumerate(output_cleaned): 
    print(slide_tag)
    channels = ["AF750", "Cy5", "Cy3", "AF488", "DAPI"]

    FILENAME = ["Base 1_c",
                "Base 2_c",
                "Base 3_c",
                "Base 4_c",
                "Base 5_c"]

    # codebook in csv (column 1: code, column2-x: color code in each round)
    # AVOID comma in gene names (column 1)
    codebook_csv = 'H:/codebook.csv'
    DO_decorators = ["Cy5", "AF750", "Cy3", "AF488"]
    CHORDER = [channels.index(i) for i in DO_decorators]

    # tile position and metadata
    # TODO: read metadata using bio-format
    pixelscale = 0.1625
    tilepos_xy_csv = 'I:/'+slide_tag+'/preprocessing/ReslicedTiles/tilepos.csv'

    # not sure why this is so important and can't be read from image itself...
    tilesz = 2000
    SHAPE = {Axes.Y: tilesz, Axes.X: tilesz}
    num_tiles = len(os.listdir('I:/'+slide_tag+'/preprocessing/ReslicedTiles/Base 1_c-1'))

    # file location
    input_dir = 'I:/'+slide_tag+'/preprocessing/ReslicedTiles'
    output_dir = 'H:/'+slide_tag+'/SpaceTX_format'

    # hakuna matata
    if not os.path.isdir(output_dir):
        try:
            os.mkdir(output_dir)
        except:
            os.makedirs(output_dir)

    tilexy = get_tilepos(tilepos_xy_csv)


    write_experiment_json(
        path=output_dir, fov_count=num_tiles, tile_format=ImageFormat.TIFF,
        primary_image_dimensions={
            Axes.ROUND: 5,
            Axes.CH: 4,
            Axes.ZPLANE: 1,
        },
        aux_name_to_dimensions={
            'nuclei': {
                Axes.ROUND: 1,
                Axes.CH: 1,
                Axes.ZPLANE: 1,
            },
            # 'dots': {
            #     Axes.ROUND: 1,
            #     Axes.CH: 1,
            #     Axes.ZPLANE: 1,
            # },
        },
        primary_tile_fetcher=ISS2DPrimaryTileFetcher(input_dir),
        aux_tile_fetcher={
            'nuclei': ISS2DAuxTileFetcher(input_dir, FILENAME[0] + "-5/tile"),
            # 'dots': ISS2DAuxTileFetcher(input_dir, FILENAME[0] + "-5/tile"),
        },
        postprocess_func=add_codebook,
        default_shape=SHAPE
    )

    make_codebook_json(codebook_csv)


    # why the fuck do they give full path now
    from shutil import copyfile
    # folder = r"H:\HCA\HCA_09b_mouse_120genes\STARFISH\smallTiles"
    ls = os.listdir(output_dir)
    os.mkdir(os.path.join(output_dir, "originaljsons"))
    for file in ls:
        if file[-4:] == 'json':
            copyfile(os.path.join(output_dir, file), os.path.join(output_dir, "originaljsons", file))
            with open(os.path.join(output_dir, "originaljsons", file), 'r') as fr:
                with open(os.path.join(output_dir, file), 'w') as fw:
                    for line in fr:
                        fw.write("%s" % line.replace(output_dir.replace('\\', '\\\\') + '\\\\', ''))



10A
10B
10C
10D
10E
14A
14B
14C
14D
18A
18B
18C


25